In [61]:
import pyspark
from pyspark.sql import SparkSession

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

In [11]:
pyspark.__version__

'3.4.0'

In [12]:
pyspark.__file__

'/usr/local/Cellar/apache-spark/3.4.0/libexec/python/pyspark/__init__.py'

In [14]:
df.write.parquet('zones_test', mode='overwrite')

In [15]:
!head taxi+_zone_lookup.csv

In [16]:
!ls -lh

total 72
-rw-r--r--  1 arunsuresh  staff    16K Apr 27 10:07 Untitled.ipynb
-rw-r--r--  1 arunsuresh  wheel    12K Aug 17  2016 taxi+_zone_lookup.csv
-rw-r--r--  1 arunsuresh  wheel   2.1K Apr 27 10:15 test.ipynb
drwxr-xr-x@ 6 arunsuresh  staff   192B Apr 27 10:16 zones_test


In [18]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2023-04-27 10:18:16--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.58, 108.138.245.96, 108.138.245.225, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: 'fhvhv_tripdata_2021-01.parquet'

fhvhv_tripdata_2021 100%[===================>] 294.61M  2.26MB/s    in 2m 11s  

2023-04-27 10:20:27 (2.25 MB/s) - 'fhvhv_tripdata_2021-01.parquet' saved [308924937/308924937]



In [19]:
!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet


In [48]:
# df_pandas = \
# pq.read_table('fhvhv_tripdata_2021-01.parquet')\
# .to_pandas(safe=False)

In [87]:
table = \
pq.read_table("fhvhv_tripdata_2021-01.parquet")

df_pandas = table.filter(
    pc.less_equal(table["dropoff_datetime"],
    pa.scalar(pd.Timestamp.max))
).to_pandas()

In [88]:
df_pandas = df_pandas.iloc[:1001, ]
# df_pandas = \
# df_pandas.loc[:, ['hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'shared_request_flag']]


In [89]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [85]:
from pyspark.sql import types

In [99]:
schema = types.StructType(
    [
        types.StructField('hvfhs_license_num', types.StringType(), True), types.StructField('dispatching_base_num', types.StringType(), True), types.StructField('originating_base_num', types.StringType(), True), types.StructField('request_datetime', types.TimestampType(), True), types.StructField('on_scene_datetime', types.TimestampType(), True), types.StructField('pickup_datetime', types.TimestampType(), True), types.StructField('dropoff_datetime', types.TimestampType(), True), types.StructField('PULocationID', types.LongType(), True), types.StructField('DOLocationID', types.LongType(), True), types.StructField('trip_miles', types.DoubleType(), True), types.StructField('trip_time', types.LongType(), True), types.StructField('base_passenger_fare', types.DoubleType(), True), types.StructField('tolls', types.DoubleType(), True), types.StructField('bcf', types.DoubleType(), True), types.StructField('sales_tax', types.DoubleType(), True), types.StructField('congestion_surcharge', types.DoubleType(), True), types.StructField('airport_fee', types.DoubleType(), True), types.StructField('tips', types.DoubleType(), True), types.StructField('driver_pay', types.DoubleType(), True), types.StructField('shared_request_flag', types.StringType(), True), types.StructField('shared_match_flag', types.StringType(), True), types.StructField('access_a_ride_flag', types.StringType(), True), types.StructField('wav_request_flag', types.StringType(), True), types.StructField('wav_match_flag', types.StringType(), True)
    ]
)

In [100]:
# df = spark.read \
#     .schema(schema) \
#     .csv('fhvhv_tripdata_2021-01.csv')

df = spark.read \
    .schema(schema) \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [103]:
# df.show()

In [104]:
# df.head(10)

In [106]:
df = df.repartition(24)

In [108]:
# df.write.parquet('fhvhv/2021/01', mode='overwrite')

df.write.parquet('fhvhv/2021/01')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/arunsuresh/Documents/arun-data-eng-zoomcamp/week_5/code/notebooks/fhvhv/2021/01 already exists. Set mode as "overwrite" to overwrite the existing path.